# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import sys
import datetime
import re
import joblib
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ddeen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddeen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_data.db')
df = pd.read_sql('SELECT * FROM messages', engine)

# Get output variables
df.set_index('id', inplace=True)
df.drop(columns=['original'], inplace=True)
X = df['message']
Y = df.drop(columns=['message','genre'])
category_names = Y.columns

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    # Convert to lowercase
    text = text.lower() 

    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 

    # tokenize
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print('Training model...')
start = datetime.datetime.now()
pipeline.fit(X_train, Y_train)
print(f'   DONE in {datetime.datetime.now()-start}')

Training model...
   DONE in 0:06:22.639743


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
y_pred = pipeline.predict(X_test)
Y_pred = Y_test.copy()
Y_pred[:] = y_pred

In [22]:
df_result = pd.DataFrame(columns=['f1-score', 'precision', 'recall'])
for label in category_names:
    classification_rep = classification_report(Y_test[label], Y_pred[label], output_dict=True)
    df_result.loc[label, ['f1-score', 'precision', 'recall']] = classification_rep['weighted avg']
df_result.mean()

f1-score     0.931731
precision    0.936676
recall       0.945576
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__min_df': [0.1, 1],
    'vect__max_df': [0.7, 1],
    # 'clf__estimator__n_estimators' = [100, 200, 500],
    'clf__estimator__max_depth': [5, 15, 25],
    # 'clf__estimator__min_samples_split' = [2, 5, 10, 15, 100],
    'clf__estimator__min_samples_leaf': [1, 2, 5],

}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
pipeline_tuned = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.7, min_df=1, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(max_depth=5, min_samples_leaf=1)))
])

print('Training model...')
start = datetime.datetime.now()
pipeline_tuned.fit(X_train, Y_train)
print(f'   DONE in {datetime.datetime.now()-start}')

y_pred = pipeline_tuned.predict(X_test)
Y_pred = Y_test.copy()
Y_pred[:] = y_pred

df_result = pd.DataFrame(columns=['f1-score', 'precision', 'recall'])
for label in category_names:
    classification_rep = classification_report(Y_test[label], Y_pred[label], output_dict=True)
    df_result.loc[label, ['f1-score', 'precision', 'recall']] = classification_rep['weighted avg']
df_result.mean()

Training model...
   DONE in 0:00:41.344912


f1-score     0.891707
precision    0.880612
recall       0.925161
dtype: float64

### 7.2 Try changing parameters manually
I tried all the improved parameters: faster, but decrease performance (< 0.9).

So I tried to remove the clf ones, because I think they are the cause

In [12]:
pipeline_tuned = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.7, min_df=1, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

print('Training model...')
start = datetime.datetime.now()
pipeline_tuned.fit(X_train, Y_train)
print(f'   DONE in {datetime.datetime.now()-start}')

y_pred = pipeline_tuned.predict(X_test)
Y_pred = Y_test.copy()
Y_pred[:] = y_pred

df_result = pd.DataFrame(columns=['f1-score', 'precision', 'recall'])
for label in category_names:
    classification_rep = classification_report(Y_test[label], Y_pred[label], output_dict=True)
    df_result.loc[label, ['f1-score', 'precision', 'recall']] = classification_rep['weighted avg']
df_result.mean()

Training model...
   DONE in 0:19:36.428327


f1-score     0.933249
precision    0.937577
recall       0.946733
dtype: float64

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
# Gradient Boosting
pipeline_gdboost = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
])

print('Training model...')
start = datetime.datetime.now()
pipeline_gdboost.fit(X_train, Y_train)
print(f'   DONE in {datetime.datetime.now()-start}')

y_pred = pipeline_gdboost.predict(X_test)
Y_pred = Y_test.copy()
Y_pred[:] = y_pred

df_result = pd.DataFrame(columns=['f1-score', 'precision', 'recall'])
for label in category_names:
    classification_rep = classification_report(Y_test[label], Y_pred[label], output_dict=True)
    df_result.loc[label, ['f1-score', 'precision', 'recall']] = classification_rep['weighted avg']
df_result.mean()

Training model...


ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.

In [28]:
# Ada Boost
pipeline_adabo = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.7, min_df=1, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

print('Training model...')
start = datetime.datetime.now()
pipeline_adabo.fit(X_train, Y_train)
print(f'   DONE in {datetime.datetime.now()-start}')

y_pred = pipeline_adabo.predict(X_test)
Y_pred = Y_test.copy()
Y_pred[:] = y_pred

df_result = pd.DataFrame(columns=['f1-score', 'precision', 'recall'])
for label in category_names:
    classification_rep = classification_report(Y_test[label], Y_pred[label], output_dict=True)
    df_result.loc[label, ['f1-score', 'precision', 'recall']] = classification_rep['weighted avg']
df_result.mean()

Training model...
   DONE in 0:04:40.225699


f1-score     0.940315
precision    0.939516
recall       0.946888
dtype: float64

In [29]:
# Ada Boost
pipeline_adabo = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

print('Training model...')
start = datetime.datetime.now()
pipeline_adabo.fit(X_train, Y_train)
print(f'   DONE in {datetime.datetime.now()-start}')

y_pred = pipeline_adabo.predict(X_test)
Y_pred = Y_test.copy()
Y_pred[:] = y_pred

df_result = pd.DataFrame(columns=['f1-score', 'precision', 'recall'])
for label in category_names:
    classification_rep = classification_report(Y_test[label], Y_pred[label], output_dict=True)
    df_result.loc[label, ['f1-score', 'precision', 'recall']] = classification_rep['weighted avg']
df_result.mean()

Training model...
   DONE in 0:05:09.917549


f1-score     0.940327
precision    0.939523
recall       0.946904
dtype: float64

### 9. Export your model as a pickle file

In [ ]:
joblib.dump(pipeline,  open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.